In [3]:
import qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_state_qsphere
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.visualization import array_to_latex
from qiskit.visualization import plot_distribution
from qiskit import transpile


import numpy as np
from numpy import sqrt

In [4]:
!pip install --quiet qiskit qiskit-ibm-runtime qiskit-aer qiskit-optimization dotenv qiskit-algorithms

In [5]:
qiskit.__version__

'2.1.2'

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save your API key and crn and have access to the quantum computers
your_api_key = "0OVe_qXjNb8l7DBhGIr5kZsnHslVTBZF0xkmpaK-EjGu"
your_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/15524aa98e0a4443b31d10fcc66c85b7:2b6de007-6ec0-499d-b370-361e9f0b70c4::"

QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token=your_api_key,
    instance=your_crn,
    overwrite=True
)                            

In [ ]:
def run_knapsack(budget: int, impact: list[int], costs: list[int], backend_name: str = "aer") -> list[int]:
    """
    Solve knapsack with QAOA or classical solver.
    """
    from qiskit_optimization.applications import Knapsack
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
    from qiskit_algorithms.optimizers import COBYLA

    # --- Build problem ---
    prob = Knapsack(values=impact, weights=costs, max_weight=budget)
    qp = prob.to_quadratic_program()
    print("Quadratic Program:")
    print(qp.prettyprint())

    # --- Classical baseline ---
    meo_classical = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
    result_classical = meo_classical.solve(qp)
    print("\nClassical (NumPyMinimumEigensolver):")
    print(result_classical.prettyprint())
    print("solution:", prob.interpret(result_classical))

    # --- Choose sampler ---
    if backend_name == "aer":
        from qiskit_aer.primitives import Sampler
        sampler = Sampler()
    else:
        from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
        service = QiskitRuntimeService()
        backend = service.backend(backend_name)
        sampler = Sampler(mode=backend)

    # --- Quantum QAOA ---
    #qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=100), reps=1)
    #ansatz=qaoa.ansatz
    #pm = transpile(ansatz, backend=backend)
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=100), reps=1)
    meo_qaoa = MinimumEigenOptimizer(qaoa)
    #qaoa.ansatz = pm.run(ansatz)
    #meo_qaoa = MinimumEigenOptimizer(pm)
    #print(meo_qaoa)
    result_qaoa = meo_qaoa.solve(qp)
    print(result_qaoa)
    print(f"\nQuantum (QAOA on {backend_name}):")
    print(result_qaoa.prettyprint())
    print("solution:", prob.interpret(result_qaoa))

    return prob.interpret(result_qaoa)

print("Final:", run_knapsack(10, [1,2,3,4,5], [4,5,6,7,8], backend_name="ibm_torino"))

Quadratic Program:
Problem name: Knapsack

Maximize
  0.01*x_0 + 0.02*x_1 + 0.03*x_2 + 0.04*x_3 + 0.05*x_4

Subject to
  Linear constraints (1)
    0.04*x_0 + 0.05*x_1 + 0.06*x_2 + 0.07*x_3 + 0.08*x_4 <= 1  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4



QiskitOptimizationError: 'Incompatible problem: Can not convert inequality constraints to equality constraint because                     float coefficients are in constraints. '

In [18]:
def run_knapsack(budget: int, impact: list[int], costs: list[int], backend_name: str = "aer") -> list[int]:
    """
    Solve knapsack with QAOA or classical solver.
    """
    from qiskit_optimization.applications import Knapsack
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
    from qiskit_algorithms.optimizers import COBYLA

    # --- Scale values (impacts) to [0, 1] ---
    max_val = max(impact) if max(impact) > 0 else 1
    scaled_impact = [v / max_val for v in impact]

    print("Original impacts:", impact)
    print("Scaled impacts:", scaled_impact)

    # --- Build problem ---
    prob = Knapsack(values=scaled_impact, weights=costs, max_weight=budget)
    qp = prob.to_quadratic_program()
    print("Quadratic Program:")
    print(qp.prettyprint())

    # --- Classical baseline ---
    meo_classical = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
    result_classical = meo_classical.solve(qp)
    print("\nClassical (NumPyMinimumEigensolver):")
    print(result_classical.prettyprint())
    print("solution:", prob.interpret(result_classical))

    # --- Choose sampler ---
    if backend_name == "aer":
        from qiskit_aer.primitives import Sampler
        sampler = Sampler()
    else:
        from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
        service = QiskitRuntimeService()
        backend = service.backend(backend_name)
        sampler = Sampler(mode=backend)

    # --- Quantum QAOA ---
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=100), reps=1)
    meo_qaoa = MinimumEigenOptimizer(qaoa)
    result_qaoa = meo_qaoa.solve(qp)
    print(result_qaoa)
    print(f"\nQuantum (QAOA on {backend_name}):")
    print(result_qaoa.prettyprint())
    print("solution:", prob.interpret(result_qaoa))

    return prob.interpret(result_qaoa)

print("Final:", run_knapsack(10, [1,2,3,4,5], [4,5,6,7,8], backend_name="ibm_torino"))


Original impacts: [1, 2, 3, 4, 5]
Scaled impacts: [0.2, 0.4, 0.6, 0.8, 1.0]
Quadratic Program:
Problem name: Knapsack

Maximize
  0.2*x_0 + 0.4*x_1 + 0.6*x_2 + 0.8*x_3 + x_4

Subject to
  Linear constraints (1)
    4*x_0 + 5*x_1 + 6*x_2 + 7*x_3 + 8*x_4 <= 10  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4


Classical (NumPyMinimumEigensolver):
objective function value: 1.0
variable values: x_0=0.0, x_1=0.0, x_2=0.0, x_3=0.0, x_4=1.0
status: SUCCESS
solution: [4]


c:\Users\tayin\.conda\envs\cwq\Lib\site-packages\qiskit_optimization\minimum_eigensolvers\qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


IBMInputValueError: 'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (-366.491965661209) outside of this range has been requested; via parameter value(s) γ[0]=-2.8632184817281954, substituted in parameter expression 128*γ[0].'

In [19]:
def run_knapsack(budget: int, impact: list[int], costs: list[int], backend_name: str = "aer") -> list[int]:
    """
    Solve knapsack with QAOA or classical solver.
    """
    from qiskit_optimization.applications import Knapsack
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
    from qiskit_algorithms.optimizers import COBYLA

    # --- Scale values (impacts) to [0, 1] ---
    max_val = max(impact) if max(impact) > 0 else 1
    scaled_impact = [v / max_val for v in impact]

    # --- Scale weights and budget to [0, 1] ---
    max_cost = max(costs) if max(costs) > 0 else 1
    scaled_costs = [w / max_cost for w in costs]
    scaled_budget = budget / max_cost

    print("Original impacts:", impact)
    print("Scaled impacts:", scaled_impact)
    print("Original costs:", costs)
    print("Scaled costs:", scaled_costs)
    print("Original budget:", budget)
    print("Scaled budget:", scaled_budget)

    # --- Build problem ---
    prob = Knapsack(values=scaled_impact, weights=scaled_costs, max_weight=scaled_budget)
    qp = prob.to_quadratic_program()
    print("Quadratic Program:")
    print(qp.prettyprint())

    # --- Classical baseline ---
    meo_classical = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
    result_classical = meo_classical.solve(qp)
    print("\nClassical (NumPyMinimumEigensolver):")
    print(result_classical.prettyprint())
    print("solution:", prob.interpret(result_classical))

    # --- Choose sampler ---
    if backend_name == "aer":
        from qiskit_aer.primitives import Sampler
        sampler = Sampler()
    else:
        from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
        service = QiskitRuntimeService()
        backend = service.backend(backend_name)
        sampler = Sampler(mode=backend)

    # --- Quantum QAOA ---
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=100), reps=1)
    meo_qaoa = MinimumEigenOptimizer(qaoa)
    result_qaoa = meo_qaoa.solve(qp)
    print(result_qaoa)
    print(f"\nQuantum (QAOA on {backend_name}):")
    print(result_qaoa.prettyprint())
    print("solution:", prob.interpret(result_qaoa))

    return prob.interpret(result_qaoa)

print("Final:", run_knapsack(10, [1,2,3,4,5], [4,5,6,7,8], backend_name="ibm_torino"))


Original impacts: [1, 2, 3, 4, 5]
Scaled impacts: [0.2, 0.4, 0.6, 0.8, 1.0]
Original costs: [4, 5, 6, 7, 8]
Scaled costs: [0.5, 0.625, 0.75, 0.875, 1.0]
Original budget: 10
Scaled budget: 1.25
Quadratic Program:
Problem name: Knapsack

Maximize
  0.2*x_0 + 0.4*x_1 + 0.6*x_2 + 0.8*x_3 + x_4

Subject to
  Linear constraints (1)
    0.5*x_0 + 0.625*x_1 + 0.75*x_2 + 0.875*x_3 + x_4 <= 1.25  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4



QiskitOptimizationError: 'Incompatible problem: Can not convert inequality constraints to equality constraint because                     float coefficients are in constraints. '

In [20]:
def run_knapsack(budget: int, impact: list[int], costs: list[int], backend_name: str = "aer") -> list[int]:
    """
    Solve knapsack with QAOA or classical solver.
    """
    from qiskit_optimization.applications import Knapsack
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
    from qiskit_algorithms.optimizers import COBYLA

    # --- Scale values (impacts) to [0, 1] ---
    max_val = max(impact) if max(impact) > 0 else 1
    scaled_impact = [v / max_val for v in impact]

    # --- Scale weights and budget to [0, 1] ---
    max_cost = max(costs) if max(costs) > 0 else 1
    scaled_costs = [w / max_cost for w in costs]
    scaled_budget = budget / max_cost

    print("Original impacts:", impact)
    print("Scaled impacts:", scaled_impact)
    print("Original costs:", costs)
    print("Scaled costs:", scaled_costs)
    print("Original budget:", budget)
    print("Scaled budget:", scaled_budget)

    # --- Build problem ---
    prob = Knapsack(values=scaled_impact, weights=scaled_costs, max_weight=scaled_budget)
    qp = prob.to_quadratic_program()
    print("Quadratic Program:")
    print(qp.prettyprint())

    # --- Classical baseline ---
    meo_classical = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
    result_classical = meo_classical.solve(qp)
    print("\nClassical (NumPyMinimumEigensolver):")
    print(result_classical.prettyprint())
    print("solution:", prob.interpret(result_classical))

    # --- Choose sampler ---
    if backend_name == "aer":
        from qiskit_aer.primitives import Sampler
        sampler = Sampler()
    else:
        from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
        service = QiskitRuntimeService()
        backend = service.backend(backend_name)
        sampler = Sampler(mode=backend)

    # --- Quantum QAOA ---
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=100), reps=1)
    meo_qaoa = MinimumEigenOptimizer(qaoa)
    result_qaoa = meo_qaoa.solve(qp)
    print(result_qaoa)
    print(f"\nQuantum (QAOA on {backend_name}):")
    print(result_qaoa.prettyprint())
    print("solution:", prob.interpret(result_qaoa))

    return prob.interpret(result_qaoa)

print("Final:", run_knapsack(10, [1,2,3,4,5], [4,5,6,7,8], backend_name="ibm_torino"))


Original impacts: [1, 2, 3, 4, 5]
Scaled impacts: [0.2, 0.4, 0.6, 0.8, 1.0]
Original costs: [4, 5, 6, 7, 8]
Scaled costs: [0.5, 0.625, 0.75, 0.875, 1.0]
Original budget: 10
Scaled budget: 1.25
Quadratic Program:
Problem name: Knapsack

Maximize
  0.2*x_0 + 0.4*x_1 + 0.6*x_2 + 0.8*x_3 + x_4

Subject to
  Linear constraints (1)
    0.5*x_0 + 0.625*x_1 + 0.75*x_2 + 0.875*x_3 + x_4 <= 1.25  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4



QiskitOptimizationError: 'Incompatible problem: Can not convert inequality constraints to equality constraint because                     float coefficients are in constraints. '

Original impacts: [1, 2, 3, 4, 5]
Original costs: [4, 5, 6, 7, 8]
Rescaled impacts: [1, 2, 4, 5, 6]
Rescaled costs: [5, 6, 8, 9, 10]
Rescaled budget: 12
Quadratic Program:
Problem name: Knapsack

Maximize
  x_0 + 2*x_1 + 4*x_2 + 5*x_3 + 6*x_4

Subject to
  Linear constraints (1)
    5*x_0 + 6*x_1 + 8*x_2 + 9*x_3 + 10*x_4 <= 12  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4


Classical (NumPyMinimumEigensolver):
objective function value: 6.0
variable values: x_0=0.0, x_1=0.0, x_2=0.0, x_3=0.0, x_4=1.0
status: SUCCESS
solution: [4]


c:\Users\tayin\.conda\envs\cwq\Lib\site-packages\qiskit_optimization\minimum_eigensolvers\qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


IBMInputValueError: 'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (3583.6626513130313) outside of this range has been requested; via parameter value(s) γ[0]=5.89418199229117, substituted in parameter expression 608*γ[0].'

In [22]:
service = QiskitRuntimeService()
service.saved_accounts()

{'default-ibm-cloud': {'channel': 'ibm_cloud',
  'url': 'https://cloud.ibm.com',
  'token': '0OVe_qXjNb8l7DBhGIr5kZsnHslVTBZF0xkmpaK-EjGu',
  'instance': 'crn:v1:bluemix:public:quantum-computing:us-east:a/15524aa98e0a4443b31d10fcc66c85b7:2b6de007-6ec0-499d-b370-361e9f0b70c4::',
  'verify': True,
  'private_endpoint': False}}

In [38]:
service.backends()

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]